<a href="https://colab.research.google.com/github/ZohanaZuthi/Spam-Detection-using-BERT/blob/main/Spam_detection_using_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text


In [6]:
import pandas as pd
df =pd.read_csv('spam.csv')
df.head(5)

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [7]:
df.groupby('Category').describe()

Message                                                            \
           count unique                                                top   
Category                                                                     
ham         4825   4516                             Sorry, I'll call later   
spam         747    641  Please call our customer service representativ...   

               
         freq  
Category       
ham        30  
spam        4

here the data is mostly of ham so it is an imbalance data. So u need to handle imbalance data

In [8]:
preprocess_url = "https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3"
encoder_url = "https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4"

In [9]:
df.groupby('Category').describe()

Message                                                            \
           count unique                                                top   
Category                                                                     
ham         4825   4516                             Sorry, I'll call later   
spam         747    641  Please call our customer service representativ...   

               
         freq  
Category       
ham        30  
spam        4

In [10]:
df['Category'].value_counts()

,count
Category,
ham,4825
spam,747


In [11]:
747/4825

0.15481865284974095

There is definately imbalance in data


In [15]:
df_ham=df[df['Category']=='ham']

In [17]:
df_spam=df[df['Category']=='spam']

In [18]:
df_ham_downsampled=df_ham.sample(df_spam.shape[0])

In [20]:
df_balanced=pd.concat([df_spam,df_ham_downsampled])

In [21]:
df_balanced['Category'].value_counts()

,count
Category,
spam,747
ham,747


In [23]:
df_balanced['spam']=df_balanced['Category'].apply(lambda x: 1 if x=='spam' else 0)
df_balanced.sample(5)

,Category,Message,spam
4162,spam,Had your mobile 11 months or more? U R entitle...,1
1745,spam,Someone has conacted our dating service and en...,1
3828,spam,Congratulations U can claim 2 VIP row A Ticket...,1
2208,ham,Usually the body takes care of it buy making s...,0
2124,spam,#ERROR!,1


In [24]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(df_balanced['Message'],df_balanced['spam'],stratify=df_balanced['spam'])

In [25]:
bert_preprocess = hub.KerasLayer(
    "https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3"
)
bert_encoder = hub.KerasLayer(
    "https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3",
    trainable=True  # We want to fine-tune BERT weights
)

In [26]:
def get_sentence_embeding(sentence):
  preprocessed_text=bert_preprocess(sentence)
  return bert_encoder(preprocessed_text)['pooled_output']
get_sentence_embeding(['500$ discount, hurry up','Bhavin, are you up for a volleybal game tomorrow?'])


<tf.Tensor: shape=(2, 768), dtype=float32, numpy=
array([[-0.7734204 , -0.50137717, -0.8073748 , ..., -0.60576576,
        -0.7409629 ,  0.8849831 ],
       [-0.87208354, -0.50543964, -0.94446665, ..., -0.85847497,
        -0.7174535 ,  0.88082975]], dtype=float32)>

In [27]:
e=get_sentence_embeding(
    [
        "banana","grapes","mango","jeff bezos","elon musk","bill gates"
    ]
)

In [28]:
e

<tf.Tensor: shape=(6, 768), dtype=float32, numpy=
array([[-0.760692  , -0.14219385,  0.49604586, ...,  0.42165312,
        -0.532214  ,  0.8031219 ],
       [-0.86023223, -0.2124296 ,  0.49156848, ...,  0.39797997,
        -0.60506296,  0.8447167 ],
       [-0.7128862 , -0.15463911,  0.384017  , ...,  0.3527875 ,
        -0.5099134 ,  0.734741  ],
       [-0.82533467, -0.35550597, -0.5906975 , ..., -0.01613778,
        -0.61417586,  0.8723029 ],
       [-0.75041336, -0.2681262 , -0.26689708, ...,  0.02839353,
        -0.5938098 ,  0.7974988 ],
       [-0.78544426, -0.2994976 ,  0.41027158, ...,  0.52225286,
        -0.49573594,  0.8150757 ]], dtype=float32)>

In [33]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_similarity([e[0]],[e[1]])

array([[0.9911088]], dtype=float32)

In [34]:
cosine_similarity([e[3]],[e[1]])

array([[0.8773976]], dtype=float32)